# Foundations of Computer Science - progetto 2022/23

- Mattia Birti [897092] 
- Alberto Porrini [826306]
- Gloria Longo [864579]

You have to work on the Dogs adoptions dataset:
*  [Dogs](https://github.com/mat1218B/ProjectOfComputerScience/blob/main/adoptions/dogs.csv)
*  [Dog Travel](https://github.com/mat1218B/ProjectOfComputerScience/blob/main/adoptions/dogTravel.csv)
*  [NST-EST2021-POP](https://github.com/mat1218B/ProjectOfComputerScience/blob/main/adoptions/NST-EST2021-POP.csv)

### Notes

1. It is mandatory to use GitHub for developing the project.
1. The project must be a jupyter notebook.
1. There is no restriction on the libraries that can be used, nor on the Python version.
1. All questions on the project must be asked in a public channel on Zulip.
1. At most 3 students can be in each group. You must create the groups by yourself.
1. You do not have to send me the project before the discussion.s

## Libraries

In [1]:
import pandas as pd

### Import data from GitHub

#### Dogs.csv

In [2]:
dogs = pd.read_csv("https://raw.githubusercontent.com/mat1218B/ProjectOfComputerScience/main/adoptions/dogs.csv", sep=",", encoding='latin-1')

In [3]:
dogs.shape

(58180, 37)

In [4]:
dogs.head()

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost
0,46042150,NV163,https://www.petfinder.com/dog/harley-46042150/...,Dog,Dog,American Staffordshire Terrier,Mixed Breed,True,False,White / Cream,...,Las Vegas,NV,89147,US,89009,2019-09-20,Dog,Harley is not sure how he wound up at shelter ...,70,124.81
1,46042002,NV163,https://www.petfinder.com/dog/biggie-46042002/...,Dog,Dog,Pit Bull Terrier,Mixed Breed,True,False,Brown / Chocolate,...,Las Vegas,NV,89147,US,89009,2019-09-20,Dog,6 year old Biggie has lost his home and really...,49,122.07
2,46040898,NV99,https://www.petfinder.com/dog/ziggy-46040898/n...,Dog,Dog,Shepherd,NaN,False,False,Brindle,...,Mesquite,NV,89027,US,89009,2019-09-20,Dog,Approx 2 years old.\n Did I catch your eye? I ...,87,281.51
3,46039877,NV202,https://www.petfinder.com/dog/gypsy-46039877/n...,Dog,Dog,German Shepherd Dog,NaN,False,False,NaN,...,Pahrump,NV,89048,US,89009,2019-09-20,Dog,NaN,62,145.83
4,46039306,NV184,https://www.petfinder.com/dog/theo-46039306/nv...,Dog,Dog,Dachshund,NaN,False,False,NaN,...,Henderson,NV,89052,US,89009,2019-09-20,Dog,Theo is a friendly dachshund mix who gets alon...,93,241.09


#### DogTravel.csv

In [5]:
dogTravel = pd.read_csv("https://raw.githubusercontent.com/mat1218B/ProjectOfComputerScience/main/adoptions/dogTravel.csv", sep=",", encoding='latin-1')

In [6]:
dogTravel.shape

(6194, 9)

In [7]:
dogTravel.head()

,index,id,contact_city,contact_state,description,found,manual,remove,still_there
0,0,44520267,Anoka,MN,Boris is a handsome mini schnauzer who made hi...,Arkansas,NaN,NaN,NaN
1,1,44698509,Groveland,FL,Duke is an almost 2 year old Potcake from Abac...,Abacos,Bahamas,NaN,NaN
2,2,45983838,Adamstown,MD,Zac Woof-ron is a heartthrob movie star lookin...,Adam,Maryland,NaN,NaN
3,3,44475904,Saint Cloud,MN,~~Came in to the shelter as a transfer from an...,Adaptil,NaN,True,NaN
4,4,43877389,Pueblo,CO,Palang is such a sweetheart. She loves her peo...,Afghanistan,NaN,NaN,NaN


#### NST-EST.csv

In [8]:
nstest = pd.read_csv("https://raw.githubusercontent.com/mat1218B/ProjectOfComputerScience/main/adoptions/NST-EST2021-POP.csv", header=None, sep=",", encoding='latin-1')

In [9]:
nstest.shape

(51, 2)

In [44]:
nstest.head()

,0,1
0,Alabama,5.024.279
1,Alaska,733.391
2,Arizona,7.151.502
3,Arkansas,3.011.524
4,California,39.538.223


## 1. Extract all dogs with status that is not adoptable

## 2. For each (primary) breed, determine the number of dogs

In [10]:
dogs.groupby('breed_primary').count()['id']

breed_primary
Affenpinscher                         17
Afghan Hound                           4
Airedale Terrier                      19
Akbash                                 3
Akita                                181
                                    ... 
Wirehaired Pointing Griffon            1
Wirehaired Terrier                    60
Xoloitzcuintli / Mexican Hairless     11
Yellow Labrador Retriever            158
Yorkshire Terrier                    360
Name: id, Length: 216, dtype: int64

## 3. For each (primary) breed, determine the ratio between the number of dogs of Mixed Breed and those not of Mixed Breed. Hint: look at the secondary_breed.

Let's see all the primary breed with a groupBy:

In [11]:
nBreed = dogs.groupby('breed_primary').count()[['species','breed_secondary']]
nBreed

,species,breed_secondary
breed_primary,,
Affenpinscher,17,2
Afghan Hound,4,1
Airedale Terrier,19,9
Akbash,3,0
Akita,181,52
...,...,...
Wirehaired Pointing Griffon,1,0
Wirehaired Terrier,60,18
Xoloitzcuintli / Mexican Hairless,11,4


As we can see we can use "species" as total number of individues for this species.
Affenpischer are in total 17 dogs, 2 of those are mixed breed.

Rename colums

In [12]:
nBreed = nBreed.rename( columns = {'species':'total', 'breed_secondary':'nSecondary'})
nBreed

,total,nSecondary
breed_primary,,
Affenpinscher,17,2
Afghan Hound,4,1
Airedale Terrier,19,9
Akbash,3,0
Akita,181,52
...,...,...
Wirehaired Pointing Griffon,1,0
Wirehaired Terrier,60,18
Xoloitzcuintli / Mexican Hairless,11,4


We have to compute how many are primary breed:

In [13]:
nBreed['nPrimary'] = nBreed['total'] - nBreed['nSecondary']
nBreed

,total,nSecondary,nPrimary
breed_primary,,,
Affenpinscher,17,2,15
Afghan Hound,4,1,3
Airedale Terrier,19,9,10
Akbash,3,0,3
Akita,181,52,129
...,...,...,...
Wirehaired Pointing Griffon,1,0,1
Wirehaired Terrier,60,18,42
Xoloitzcuintli / Mexican Hairless,11,4,7


It's time to compute the ratio between Primary and Secondary Breed.

In [14]:
nBreed['ratioBreed'] = nBreed['nPrimary'] / nBreed['nSecondary']
nBreed

,total,nSecondary,nPrimary,ratioBreed
breed_primary,,,,
Affenpinscher,17,2,15,7.500000
Afghan Hound,4,1,3,3.000000
Airedale Terrier,19,9,10,1.111111
Akbash,3,0,3,inf
Akita,181,52,129,2.480769
...,...,...,...,...
Wirehaired Pointing Griffon,1,0,1,inf
Wirehaired Terrier,60,18,42,2.333333
Xoloitzcuintli / Mexican Hairless,11,4,7,1.750000


As we can see, there are many 'inf' values. It is correct because for that breed there are not any secondary breed.

## 4. For each (primary) breed, determine the earliest and the latest posted timestamp.

## 5. For each state, compute the sex imbalance, that is the difference between male and female dogs. In which state this imbalance is largest?

## 6. For each pair (age, size), determine the average duration of the stay and the average cost of stay.

Explore the dog's column names

In [15]:
dogs.columns

Index(['id', 'org_id', 'url', 'type.x', 'species', 'breed_primary',
       'breed_secondary', 'breed_mixed', 'breed_unknown', 'color_primary',
       'color_secondary', 'color_tertiary', 'age', 'sex', 'size', 'coat',
       'fixed', 'house_trained', 'declawed', 'special_needs', 'shots_current',
       'env_children', 'env_dogs', 'env_cats', 'name', 'status', 'posted',
       'contact_city', 'contact_state', 'contact_zip', 'contact_country',
       'stateQ', 'accessed', 'type.y', 'description', 'stay_duration',
       'stay_cost'],
      dtype='object')

Compute the mean of the stay duration and stay cost for the pair AGE, SIZE;

In [16]:
AgeSize = dogs.groupby(['age', 'size'])[['stay_duration','stay_cost']].mean()
AgeSize

stay_duration   stay_cost
age    size                                  
Adult  Extra Large      89.015414  232.591561
       Large            89.531943  238.661141
       Medium           89.421036  238.258977
       Small            89.407479  238.974838
Baby   Extra Large      87.032967  237.180879
       Large            89.701564  238.698827
       Medium           89.577668  237.108131
       Small            89.958291  239.083810
Senior Extra Large      88.861111  235.232361
       Large            88.984298  237.507364
       Medium           89.810052  238.514615
       Small            89.073626  238.282286
Young  Extra Large      90.586345  245.835582
       Large            90.104206  238.149506
       Medium           89.515123  239.304603
       Small            89.814275  241.540069

## 7. Find the dogs involved in at least 3 travels. Also list the breed of those dogs.

## 8. Fix the travels table so that the correct state is computed from the manual and the found fields. If manual is not missing, then it overrides what is stored in found.

## 9. For each state, compute the ratio between the number of travels and the population.

In [27]:
dogTravel

,index,id,contact_city,contact_state,description,found,manual,remove,still_there
0,0,44520267,Anoka,MN,Boris is a handsome mini schnauzer who made hi...,Arkansas,NaN,NaN,NaN
1,1,44698509,Groveland,FL,Duke is an almost 2 year old Potcake from Abac...,Abacos,Bahamas,NaN,NaN
2,2,45983838,Adamstown,MD,Zac Woof-ron is a heartthrob movie star lookin...,Adam,Maryland,NaN,NaN
3,3,44475904,Saint Cloud,MN,~~Came in to the shelter as a transfer from an...,Adaptil,NaN,True,NaN
4,4,43877389,Pueblo,CO,Palang is such a sweetheart. She loves her peo...,Afghanistan,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
6189,6189,40492179,Fairmont,WV,Please contact Pet (information@pethelpersinc....,WV,NaN,True,NaN
6190,6190,45799729,Eagle Mountain,UT,Shiny is an approximately 4-6-year-old spayed ...,Wyoming,NaN,NaN,NaN
6191,6191,34276515,Newnan,GA,Yanni is a Male Great Pyrenees that we rescue...,Yazmin,NaN,True,NaN
6192,6192,44519341,Dayton,OH,Callie is a 14 year old Chihuahua whose owner ...,Young,Ohio,NaN,NaN


In [31]:
dogTravel[dogTravel['found'] == 'Akron']

,index,id,contact_city,contact_state,description,found,manual,remove,still_there
6,6,45287347,Wooster,OH,"Tate is an adorable 2 year old, 22 pound Cocka...",Akron,Ohio,NaN,NaN
7,7,45287347,Wooster,OH,"Tate is an adorable 2 year old, 22 pound Cocka...",Akron,Ohio,NaN,NaN


In [33]:
dogTravel[dogTravel['contact_state'] == 'OH']

,index,id,contact_city,contact_state,description,found,manual,remove,still_there
6,6,45287347,Wooster,OH,"Tate is an adorable 2 year old, 22 pound Cocka...",Akron,Ohio,NaN,NaN
7,7,45287347,Wooster,OH,"Tate is an adorable 2 year old, 22 pound Cocka...",Akron,Ohio,NaN,NaN
364,364,45916456,Amelia,OH,MEATBALL is a 3-1/2 month old Great Pyrenees m...,Amelia,Ohio,NaN,NaN
365,365,45916450,Amelia,OH,LEONIDAS is a 3-1/2 month old Great Pyrenees m...,Amelia,Ohio,NaN,NaN
366,366,45916444,Amelia,OH,"FAWN is a 3-1/2 month old Great Pyrenees mix, ...",Amelia,Ohio,NaN,NaN
...,...,...,...,...,...,...,...,...,...
5485,5485,45774206,Columbus,OH,"Hello world, my name is Tyra, yes like the sup...",Trenton,Ohio,NaN,NaN
6147,6147,44403455,Columbus,OH,We really want to express how much fun and how...,Wheeling,West Virginia,NaN,NaN
6165,6165,45971387,Wooster,OH,PLEASE DO NOT USE THE PETFINDER LINK TO CONTAC...,Wooster,NaN,True,NaN
6166,6166,44662491,Wooster,OH,This adorable little fella is Sly. He is a 9 ...,Wooster,NaN,True,NaN


In [35]:
tablecity = dogTravel.groupby('contact_city').count()[['index']]
tablecity

,index
contact_city,
Abingdon,1
Ada,1
Adamstown,1
Aiken,1
Akron,1
...,...
Yuba City,1
Yukon,2
Zanesville,1


In [46]:
tablestate = dogTravel.groupby('contact_state').count()[['index']]
tablestate.shape

(45, 1)

I numeri di stati (viaggiatori) sono 45, i numeri di stati in stest sono 51, significa che ci sono 6 stati che non viaggiano 

In [29]:
nTravelForState = dogTravel.groupby('found').count()[['index']]
nTravelForState = nTravelForState.rename(columns={'index':'nTravel'})
nTravelForState

,nTravel
found,
Abacos,1
Adam,1
Adaptil,1
Afghanistan,2
Akron,2
...,...
the Guernsey County Dog,36
the Jefferson City,1
the State of California,1


In [41]:
state = nstest.rename(columns = {0:'state', 1:'nPopulation'})
state.shape

(51, 2)

We have obtained the number of travel for state.

Make a merge between two table;

In [38]:
newtab = pd.merge(state, nTravelForState, left_on='state', right_on='found')
##ci sono meno righe, gli stati in totale sono 51, significa che ci sono stati che non hanno viaggiatori
newtab

,state,nPopulation,nTravel
0,Alabama,5.024.279,322
1,Arizona,7.151.502,18
2,Arkansas,3.011.524,35
3,California,39.538.223,130
4,Colorado,5.773.714,11
5,Connecticut,3.605.944,16
6,Delaware,989.948,30
7,Florida,21.538.187,83
8,Georgia,10.711.908,134
9,Hawaii,1.455.271,2


## 10. For each dog, compute the number of days from the posted day to the day of last access.

## 11. Partition the dogs according to the number of weeks from the posted day to the day of last access.

## 12. Find for duplicates in the dogs dataset. Two records are duplicates if they have (1) same breeds and sex, and (2) they share at least 90% of the words in the description field. Extra points if you find and implement a more refined for determining if two rows are duplicates.